In [1]:
import torch

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(777)
if device == "cuda":
    torch.cuda.manual_seed_all(777)

In [ ]:
X = torch.FloatTensor([[0,0],
                       [0,1],
                       [1,0],
                       [1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

In [ ]:
linear = torch.nn.Linear(2,1,bias = True)
sigmoid = torch.nn.Sigmoid()

In [ ]:
model = torch.nn.Sequential(linear,sigmoid).to(device)

In [ ]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = 1)

In [ ]:
for step in range(10001):
    optimizer.zero_grad()
    hypothesis = model(X)

    # cost/loss function
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0:
        print(step, cost.item())

In [ ]:
with torch.no_grad():
    hypothesis = model(X)
    predicted = (hypothesis > 0.5).float()
    accuracy = (predicted == Y).float().mean()
    print('\nHypothesis: ', hypothesis.detach().cpu().numpy(), '\nCorrect: ', predicted.detach().cpu().numpy(), '\nAccuracy: ', accuracy.item())

# Multi Layer Perceptron

In [ ]:
X = torch.FloatTensor([[0,0],
                       [0,1],
                       [1,0],
                       [1,1]]).to(device)
Y = torch.FloatTensor([[0],[1],[1],[0]]).to(device)

w1 = torch.Tensor(2,2).to(device)
b1 = torch.Tensor(2).to(device)
w2 = torch.Tensor(2,1).to(device)
b2 = torch.Tensor(1).to(device)

In [ ]:
def sigmoid(x):
    return 1.0/(1.0 + torch.exp(-x))

def sigmoid_prime(x):
    return sigmoid(x) * (1-sigmoid(x))

In [ ]:
learning_rate = 0.01
for step in range(10001):
    # forward
    l1 = torch.add(torch.matmul(X,w1),b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1,w2),b2)
    Y_pred = sigmoid(l2)
    
    cost = -torch.mean(Y * torch.log(Y_pred) + (1-Y) *torch.log(1-Y_pred)) # binary cross entropy를 미분한 식
    
    # Backpropagation
    # Loss derivative
    d_Y_pred =(Y_pred - Y) / (Y_pred * (1.0 - Y_pred)+1e-7)
    
    d_l2 = d_Y_pred * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1,0,1), d_b2)
    
    d_a1 = torch.matmul(d_b2,torch.transpose(w2,0,1))
    d_l1 = d_a1 * sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X,0,1), d_b1)
    
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1,0)
    w2 = w2 - learning_rate * d_w2
    b2 = b2 - learning_rate*torch.mean(d_b2,0)
    
    if step% 1000 == 0:
        print(step,cost.item())

## XOR_nn